In [ ]:
import numpy as np
import pandas as pd
import random
import time
from itertools import product
from scipy.stats import norm, binomtest
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Lasso, Ridge

from sklearn import linear_model, svm, naive_bayes, ensemble
from sklearn.model_selection import cross_validate, train_test_split, RepeatedStratifiedKFold
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix, roc_auc_score, matthews_corrcoef

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


## Read functions

In [ ]:
%run functions_scenario1.ipynb

## Scenario 1

## Logistic regression

In [ ]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0))
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
                    # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
#                     ('RF', ensemble.RandomForestClassifier(n_estimators=200, criterion='gini', random_state=0))
                    # ('XGB', XGBClassifier(n_estimators=50, learning_rate=0.1, random_state=0))
                   ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
        'LR__C': [0.01] # [0.01, 0.05, 0.1]
        # 'Ridge__C': [0.01]
        # 'SVM__C': [0.01, 0.05, 0.1]
#         'RF__n_estimators': [50] # [10, 30, 50]
        # 'XGB__n_estimators': [50]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

# def ensemble_model_fit(data, data_pred):
#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the Logistic Regression model
#     logistic_regression = LogisticRegression(C=0.01, max_iter=200,random_state=0)

#     # Fit the Logistic Regression model on the training data
#     logistic_regression.fit(X_train, y_train)

#     # Print out the coefficients of the logistic regression model
#     print("Coefficients:", logistic_regression.coef_)
#     print("Intercept:", logistic_regression.intercept_)
#     print("Coefficients shape:", logistic_regression.coef_.shape)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = logistic_regression.predict_proba(X_test)
#     y_pred_test = logistic_regression.predict(X_test)

#     # You can also predict probabilities for the prediction data (data_pred)
#     X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1)
#     y_pred = logistic_regression.predict_proba(X_dt)

#     return y_pred

# # Call the function
# # ensemble_model_fit(data, data_pred)


### Simulation starts

In [ ]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 5
n_patient_per_plan = 5468
n_rounds = 5
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [ ]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



### Results summary

In [ ]:
rr_df = pd.DataFrame(rr_dict)
rr_df

In [ ]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

In [ ]:
# ORR for each round:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# Overall ORR:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

In [ ]:
# Highest ORR:
print(np.mean(highest_true_rr))
print(np.std(highest_true_rr))

In [ ]:
# Adaptive ORR:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

In [ ]:
# Average plan number for each round:
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# Early stopping probabilities for different reasons:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

In [ ]:
# sample size for the last round:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

In [ ]:
# Probability of better performance compared to the benchmark:
np.mean(better_chance)

In [ ]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

## Random Forest

In [ ]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            # ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0))
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
                    # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
                    ('RF', ensemble.RandomForestClassifier(criterion='gini', random_state=0))
                    # ('XGB', XGBClassifier(n_estimators=50, learning_rate=0.1, random_state=0))
                   ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
        # 'LR__C': [0.01] # [0.01, 0.05, 0.1]
        # 'Ridge__C': [0.01]
        # 'SVM__C': [0.01, 0.05, 0.1]
        'RF__n_estimators': [50] # [10, 30, 50]
        # 'XGB__n_estimators': [50]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

# def ensemble_model_fit(data, data_pred):
#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the Logistic Regression model
#     logistic_regression = LogisticRegression(C=0.01, max_iter=200,random_state=0)

#     # Fit the Logistic Regression model on the training data
#     logistic_regression.fit(X_train, y_train)

#     # Print out the coefficients of the logistic regression model
#     print("Coefficients:", logistic_regression.coef_)
#     print("Intercept:", logistic_regression.intercept_)
#     print("Coefficients shape:", logistic_regression.coef_.shape)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = logistic_regression.predict_proba(X_test)
#     y_pred_test = logistic_regression.predict(X_test)

#     # You can also predict probabilities for the prediction data (data_pred)
#     X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1)
#     y_pred = logistic_regression.predict_proba(X_dt)

#     return y_pred

# # Call the function
# # ensemble_model_fit(data, data_pred)


### Simulation starts

In [ ]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 5
n_patient_per_plan = 5468
n_rounds = 5
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [ ]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



### Results summary

In [ ]:
rr_df = pd.DataFrame(rr_dict)
rr_df

In [ ]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

In [ ]:
# Average ORR for each round:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# overall ORR:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

In [ ]:
# Adaptive learning ORR:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

In [ ]:
# Highest True RR:
mean_result = (np.mean(highest_true_rr))
std_result = (np.std(highest_true_rr))
print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

In [ ]:
# Average plan number at each round:
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

In [ ]:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

In [ ]:
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# Prob. of better performance compared to the benchmark 
np.mean(better_chance)

## XGBoost

In [ ]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            # ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0))
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
                    # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
#                     ('RF', ensemble.RandomForestClassifier(n_estimators=200, criterion='gini', random_state=0))
                    ('XGB', XGBClassifier(n_estimators=50, learning_rate=0.1, random_state=0))
                   ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
        # 'LR__C': [0.01] # [0.01, 0.05, 0.1]
        # 'Ridge__C': [0.01]
        # 'SVM__C': [0.01, 0.05, 0.1]
#         'RF__n_estimators': [50] # [10, 30, 50]
        'XGB__n_estimators': [50]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

### Simulation starts

In [ ]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 5
n_patient_per_plan = 5468
n_rounds = 5
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [ ]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



### Results summary

In [ ]:
rr_df = pd.DataFrame(rr_dict)
rr_df

In [ ]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

In [ ]:
# ORR for each round:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# Overall ORR:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

In [ ]:
# Highest True ORR:
print(np.mean(highest_true_rr))
print(np.std(highest_true_rr))

In [ ]:
# Adaptive learning ORR:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

In [ ]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

In [ ]:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

In [ ]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
np.mean(better_chance)

## Ensemble learning - 3 methods

In [ ]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0)),
                    ('RF', ensemble.RandomForestClassifier(criterion='gini', random_state=0)),
                    ('XGB', XGBClassifier(learning_rate=0.1, random_state=0))
                   ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        'LR__C': [0.05],
        'RF__n_estimators': [100],
        'XGB__n_estimators': [50]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

### Simulation starts

In [ ]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 5
n_patient_per_plan = 5468
n_rounds = 5
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [ ]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



### Results summary

In [ ]:
rr_df = pd.DataFrame(rr_dict)
rr_df

In [ ]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

In [ ]:
# orr for each round:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# overall orr:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

In [ ]:
# adaptive learning orr:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

In [ ]:
# highest true rr:
mean_result = (np.mean(highest_true_rr))
pristd_result = (np.std(highest_true_rr))

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

In [ ]:
# average plan number for each round:
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# early stopping probabilities:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

In [ ]:
# sample size for the last round:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

In [ ]:
# probability of better performance compared to the benchmark:
np.mean(better_chance)

In [ ]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

## Ensemble learning - 7 methods

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import ensemble, linear_model
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan', 'plan_response_rate', 'group_size', 'response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            ('LR', linear_model.LogisticRegression(penalty = 'none', max_iter=200, random_state=0)),
            ('Lasso', linear_model.LogisticRegression(penalty = "l1", max_iter=200, random_state=0,
                                                     solver="liblinear")),
            ('Ridge', linear_model.LogisticRegression(penalty = "l2", max_iter=200, random_state=0)),
            ('GBM', ensemble.GradientBoostingClassifier(random_state=0)),
            ('RF', ensemble.RandomForestClassifier(random_state=0)),
            ('XGB', XGBClassifier(random_state=0)),
            ('NN', MLPClassifier(random_state=0))
        ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
#     param_grid = {
#         'Lasso__C': [0.01, 0.1, 1.0],  # Regularization parameter for lasso regression
#         'Ridge__C': [0.01, 0.1, 1.0],  # Regularization parameter for ridge regression
#         'GBM__learning_rate': [0.01, 0.1, 0.5],  # Learning rate for gradient boosting machine
#         'GBM__n_estimators': [50, 100, 200],  # Number of trees for gradient boosting machine
#         'RF__n_estimators': [50, 100, 200],  # Number of trees for random forest
#         'XGB__learning_rate': [0.01, 0.1, 0.5],  # Learning rate for XGBoost
#         'XGB__n_estimators': [50, 100, 200],  # Number of trees for XGBoost
#         'NN__hidden_layer_sizes': [(50,), (100,), (50, 50)],  # Size of hidden layers for neural networks
#         'NN__alpha': [0.0001, 0.001, 0.01]  # Regularization parameter for neural networks
#     }
    param_grid = {
        'Lasso__C': [0.1],  # Regularization parameter for lasso regression
        'Ridge__C': [0.01],  # Regularization parameter for ridge regression
        'GBM__learning_rate': [0.01],  # Learning rate for gradient boosting machine
        'GBM__n_estimators': [50],  # Number of trees for gradient boosting machine
        'RF__n_estimators': [50],  # Number of trees for random forest
        'XGB__learning_rate': [0.01],  # Learning rate for XGBoost
        'XGB__n_estimators': [50],  # Number of trees for XGBoost
        'NN__hidden_layer_sizes': [(50,)],  # Size of hidden layers for neural networks
        'NN__alpha': [0.01]  # Regularization parameter for neural networks
    }

    # Create a GridSearchCV object
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
#     print("Best fit parameters:", best_params)

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size', 'response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred


In [ ]:
# ensemble_model_fit(dt_design_5, dt_design_5)

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

# def ensemble_model_fit(data, data_pred):
#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the Logistic Regression model
#     logistic_regression = LogisticRegression(C=0.01, max_iter=200,random_state=0)

#     # Fit the Logistic Regression model on the training data
#     logistic_regression.fit(X_train, y_train)

#     # Print out the coefficients of the logistic regression model
#     print("Coefficients:", logistic_regression.coef_)
#     print("Intercept:", logistic_regression.intercept_)
#     print("Coefficients shape:", logistic_regression.coef_.shape)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = logistic_regression.predict_proba(X_test)
#     y_pred_test = logistic_regression.predict(X_test)

#     # You can also predict probabilities for the prediction data (data_pred)
#     X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1)
#     y_pred = logistic_regression.predict_proba(X_dt)

#     return y_pred

# # Call the function
# # ensemble_model_fit(data, data_pred)


### Simulation starts

In [ ]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 5
n_patient_per_plan = 5468
n_rounds = 5
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [ ]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []
highest_true_rr = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    highest_true_rr.append(np.max(dt_design_5['plan_response_rate']))
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, 
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



### Results summary

In [ ]:
rr_df = pd.DataFrame(rr_dict)
rr_df

In [ ]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

In [ ]:
# orr for each round:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# overall orr:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

In [ ]:
# highest true rr:
print(np.mean(highest_true_rr))
print(np.std(highest_true_rr))

In [ ]:
# adaptive learning orr:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

In [ ]:
# average plan number for each round:
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# early stopping probabilities:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

In [ ]:
# sample size for the last round:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

In [ ]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# probability of better performance compared to the benchmark:
np.mean(better_chance)